In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

%load_ext tensorboard

In [2]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector

2022-03-04 22:20:12.835941: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-04 22:20:12.835974: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/fung/.pyenv/versions/3.8.12/envs/EFPO/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
(train_data, test_data), info = tfds.load(
    "imdb_reviews/subwords8k",
    split=(tfds.Split.TRAIN, tfds.Split.TEST),
    with_info=True,
    as_supervised=True,
)
encoder = info.features["text"].encoder

# Shuffle and pad the data.
train_batches = train_data.shuffle(1000).padded_batch(
    10, padded_shapes=((None,), ())
)
test_batches = test_data.shuffle(1000).padded_batch(
    10, padded_shapes=((None,), ())
)
train_batch, train_labels = next(iter(train_batches))

2022-03-04 22:20:14.195916: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (100)
2022-03-04 22:20:14.195961: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-OQ3QMKM): /proc/driver/nvidia/version does not exist
2022-03-04 22:20:14.196374: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-04 22:20:14.309576: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline s

In [4]:
# Create an embedding layer.
embedding_dim = 16
embedding = tf.keras.layers.Embedding(encoder.vocab_size, embedding_dim)
# Configure the embedding layer as part of a keras model.
model = tf.keras.Sequential(
    [
        embedding, # The embedding layer should be the first layer in a model.
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1),
    ]
)

# Compile model.
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Train model for one epoch.
history = model.fit(
    train_batches, epochs=1, validation_data=test_batches, validation_steps=20)

2500/2500 [==============================] - 6s 2ms/step - loss: 0.5259 - accuracy: 0.6822 - val_loss: 0.3728 - val_accuracy: 0.8700


2022-03-04 22:20:19.962716: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [7]:
# Set up a logs directory, so Tensorboard knows where to look for files.
log_dir='logs/imdb-example/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for subwords in encoder.subwords:
    f.write("{}\n".format(subwords))
  # Fill in the rest of the labels with "unknown".
  for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
    f.write("unknown #{}\n".format(unknown))


# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(model.layers[0].get_weights()[0][1:])
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

# Now run tensorboard against on log data we just saved.
%tensorboard --logdir logs/imdb-example

Reusing TensorBoard on port 6006 (pid 6404), started 0:00:44 ago. (Use '!kill 6404' to kill it.)

In [13]:
import pandas as pd

In [14]:
df=pd.read_csv("trialdataFung02.csv")

In [16]:
df.values

(880, 301)

In [21]:
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,0,-0.002882,-0.090932,0.027915,-0.021491,-0.097215,0.003197,0.044597,0.142039,0.045104,...,-0.007293,-0.105503,0.040222,-0.005296,-0.021423,-0.135647,-0.035484,-0.015481,-0.091475,-0.003674
1,1,0.060205,-0.142388,-0.053261,0.006739,0.002351,0.010120,0.004299,0.017446,0.024857,...,0.010719,-0.115145,0.003793,0.047363,-0.020292,-0.002742,-0.035765,-0.088677,-0.047915,-0.012275
2,2,-0.043575,-0.089596,-0.059855,-0.027599,0.054356,0.123403,-0.018598,-0.081314,-0.057007,...,0.019487,-0.064594,-0.017959,-0.049705,-0.062980,0.104403,0.035142,-0.015006,0.024766,-0.046359
3,3,0.126782,-0.071671,-0.020207,0.106401,-0.012710,-0.038288,0.028167,0.005505,-0.026489,...,0.074442,-0.097198,0.065517,0.039933,0.137622,0.014308,0.017518,0.005966,-0.047279,-0.037677
4,4,0.116386,-0.029600,-0.031809,0.111071,-0.017531,-0.025424,0.030436,-0.027655,-0.004751,...,0.069993,-0.110918,0.015859,0.029177,0.165497,0.025816,-0.052511,0.000903,-0.029496,-0.074084


In [20]:
# Set up a logs directory, so Tensorboard knows where to look for files.
log_dir='logs/test/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for subwords in encoder.subwords:
    f.write("{}\n".format(subwords))
  # Fill in the rest of the labels with "unknown".
  for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
    f.write("unknown #{}\n".format(unknown))


# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(df.values)
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

# Now run tensorboard against on log data we just saved.
%tensorboard --logdir logs/test

Reusing TensorBoard on port 6008 (pid 7545), started 0:00:09 ago. (Use '!kill 7545' to kill it.)